In [72]:
import pandas as pd
import nltk
import numpy as np
from functools import reduce

NOME_DO_ARQUIVO = 'estadao_noticias_eleicao.csv'
indice_invertido = {}
dados = pd.read_csv(NOME_DO_ARQUIVO)
dados = dados.replace(np.nan, '', regex=True)
dados["TF"] = None

In [75]:
def geraIndice():
    for i in range(dados.titulo.size):
        texto = dados.titulo[i] + " " + dados.conteudo[i] + " " + dados.subTitulo[i]
        palavras_divididas = nltk.word_tokenize(texto)
        dados.TF[i] = {}
        for palavra in palavras_divididas:
            palavra = palavra.lower()
            if palavra not in indice_invertido:
                indice_invertido[palavra] = set([dados.idNoticia[i]])
            else:
                indice_invertido[palavra].add(dados.idNoticia[i])
            if palavra not in dados.TF[i]:
                dados.TF[i][palavra] = 0
            dados.TF[i][palavra] += 1

In [83]:
def modeloBinario(termos):
    return list(reduce((lambda x,y: indice_invertido[x] & indice_invertido[y]), termos))

In [89]:
def getDocumentsWithTerms(terms):
    return list(reduce((lambda x,y: indice_invertido[x].union(indice_invertido[y])), terms))

In [85]:
def generateQueryTF(query):
    tf = {}
    for term in query:
        if term not in tf:
            tf[term] = 0
        tf[term] += 1
    return tf

In [110]:
def modeloTF(termos):
    documentos = getDocumentsWithTerms(termos)
    tfQuery = generateQueryTF(termos)
    documentsTF = []
    for idDoc in documentos:
        for queryTerm in tfQuery:
            docTF = dados.TF[idDoc][queryTerm] if queryTerm in dados.TF[idDoc] else 0
            qTF = tfQuery[queryTerm] if queryTerm in tfQuery else 0
            documentsTF.append((idDoc, docTF * qTF))
    documentsTF.sort(key=lambda tup: tup[1], reverse=True)
    return documentsTF

In [111]:
print(modeloTF(["senado", "brasileiro"]))

[(1004, 16), (6442, 12), (5342, 9), (6879, 9), (873, 8), (5506, 8), (5546, 8), (7387, 8), (679, 7), (749, 7), (6840, 7), (7158, 7), (8176, 7), (8341, 6), (335, 6), (5333, 6), (5394, 6), (1540, 6), (2072, 6), (6189, 6), (453, 6), (6440, 6), (6837, 6), (2754, 6), (1024, 5), (1044, 5), (5255, 5), (5334, 5), (5995, 5), (6182, 5), (6252, 5), (2472, 5), (7263, 5), (7386, 5), (332, 4), (4628, 4), (577, 4), (698, 4), (726, 4), (844, 4), (846, 4), (5134, 4), (1062, 4), (6243, 4), (2371, 4), (6636, 4), (6862, 4), (2920, 4), (7028, 4), (7110, 4), (7382, 4), (7528, 4), (7564, 4), (3772, 4), (4052, 4), (4109, 3), (8231, 3), (53, 3), (75, 3), (85, 3), (8286, 3), (8432, 3), (245, 3), (8602, 3), (413, 3), (454, 3), (455, 3), (8700, 3), (4617, 3), (4624, 3), (667, 3), (760, 3), (769, 3), (770, 3), (776, 3), (784, 3), (815, 3), (816, 3), (817, 3), (819, 3), (5020, 3), (5032, 3), (5033, 3), (940, 3), (963, 3), (5059, 3), (970, 3), (5129, 3), (1085, 3), (5331, 3), (1244, 3), (5133, 3), (5424, 3), (5428, 3